In [41]:
import json
import pandas as pd
import csv

# Ler o arquivo JSON
with open('movies.json', 'r+') as file:
    data = json.load(file)

    # Percorrer as chaves do dicionário
    for movie_title, movie_data in data.items():
        # Criar uma nova entrada no dicionário com a coluna "label" e valor "None" como string
        movie = movie_data['data']
        if movie['movie'] != None:
            movie['movie']['label'] = "None"

    # Voltar para o início do arquivo
    file.seek(0)

    # Escrever o arquivo JSON atualizado
    json.dump(data, file, indent=4)

    # Truncar o restante do arquivo, caso o novo conteúdo seja menor que o original
    file.truncate()

In [39]:
with open('movies.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Title', 'Type','CriticScore', 'Label/Studio', 'ReleaseYear', 'ReleaseMonth', 'Director', 'Genre', 'Rating', 'Runtime', 'NumOfCriticReviews'])
    
    for movie_title, movie_data in data.items():
        movie = movie_data['data']
        if((movie.get('movie'))):
            genre = "/".join([genre.strip() for genre in movie['movie']['genres']])
            director = "/".join([genre.strip() for genre in movie['movie']['director']])
            if movie['movie']['criticScore'] >= 90:
                score = "Excellent"
            elif movie['movie']['criticScore'] >= 70:
                score = "Good"
            elif movie['movie']['criticScore'] >= 40:
                score = "Mediocre"
            else:
                score = "Bad"
            date = movie['movie']['releaseDate'].split()
            month = date[0]
            writer.writerow([
                movie['movie']['title'],
                "Movie",
                score,
                movie['movie']['label'],
                movie['movie']['year'],
                month,
                director,
                genre,
                movie['movie']['rating'],
                movie['movie']['runtime'],
                movie['movie']['numOfCriticReviews']
            ])


In [40]:
pd.read_csv("movies.csv")

,Title,Type,CriticScore,Label/Studio,ReleaseYear,ReleaseMonth,Director,Genre,Rating,Runtime,NumOfCriticReviews
0,The Godfather,Movie,Bad,None,1972,March,Francis Ford Coppola,Crime,R,175 min,16
1,The Shawshank Redemption,Movie,Good,None,1994,September,Frank Darabont,Crime,R,142 min,21
2,The Dark Knight,Movie,Good,None,2008,July,Christopher Nolan,Crime,PG-13,152 min,39
3,The Lord of the Rings: The Fellowship of the Ring,Movie,Excellent,None,2001,December,Peter Jackson,Fantasy,PG-13,178 min,34
4,Pulp Fiction,Movie,Excellent,None,1994,October,Quentin Tarantino,Crime,R,154 min,24
...,...,...,...,...,...,...,...,...,...,...,...
105,The Spectacular Now,Movie,Good,None,2013,August,James Ponsoldt,Romance,R,95 min,42
106,Safety Not Guaranteed,Movie,Good,None,2012,June,Colin Trevorrow,Comedy,R,86 min,31
107,Like Crazy,Movie,Mediocre,None,2011,October,Drake Doremus,Romance,PG-13,90 min,38
108,The Art of Getting By,Movie,Bad,None,2011,June,Gavin Wiesen,Romance,PG-13,84 min,28
